# Fine-tuning Test

In [56]:
from datasets import *
import datasets
import evaluate
from evaluate import evaluator
from evaluate.visualization import radar_plot
from transformers import AutoModelForSequenceClassification, pipeline


# dataset = load_dataset('lmsys/toxic-chat', 'toxicchat0124', split="test").shuffle()
# dataset = dataset.rename_column("user_input", "text")
# dataset = dataset.rename_column("jailbreaking", "label")
# dataset = dataset.remove_columns("conv_id")
# dataset = dataset.remove_columns("model_output")
# dataset = dataset.remove_columns("human_annotation")
# dataset = dataset.remove_columns("toxicity")
# dataset = dataset.remove_columns("openai_moderation")

# dataset = load_dataset('jackhhao/jailbreak-classification', split="test").shuffle()
# dataset = dataset.rename_column("prompt", "text")
# dataset = dataset.rename_column("type", "label")
# dataset = dataset.cast_column("label", datasets.ClassLabel(names=["benign", "jailbreak"]))

# dataset = load_dataset('deepset/prompt-injections', split="test").shuffle()

dataset = load_dataset('xTRam1/safe-guard-prompt-injection', split="test").shuffle()

dataset[0]


{'text': 'It was dumb luck, his bad move had actually put him in an what position?\nOptions:\n- virtuous\n- advantageous\n- excellent\n- adventurous\n- exceptional\nStep-by-step reasoning process below:\n',
 'label': 0}

In [57]:
data = dataset
task_evaluator = evaluator("text-classification")

# 1. Pass a model name or path
eval_results = task_evaluator.compute(
    model_or_pipeline="cyrp/distilbert-base-uncased-lora-text-classification-deepset-promptinjection",
    data=data,
    input_column="text",
    label_column="label",
    label_mapping={"LABEL_0": 0, "LABEL_1": 1},
    metric=evaluate.combine(["accuracy", "recall", "precision", "f1"])
)
print(eval_results) 

c:\Users\chefb\OneDrive\Laufbahn\Hochschule\S8\bachelorarbeit\.venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'accuracy': 0.6980582524271844, 'recall': 0.49230769230769234, 'precision': 0.5228758169934641, 'f1': 0.5071315372424723, 'total_time_in_seconds': 13.701967099998, 'samples_per_second': 150.3433766090637, 'latency_in_seconds': 0.006651440339804855}
